In [ ]:
import sys
print(sys.executable)

#& "C:\Program Files\Python312\python.exe" -m pip install pdfplumber
#& "C:\Program Files\Python312\python.exe" -m pip install camelot
#& "C:\Program Files\Python312\python.exe" -m pip install camelot-py[cv] pdfplumber pandas openpyxl

c:\Program Files\Python312\python.exe


In [21]:
import os
import re
import pdfplumber
import pandas as pd
from tabula.io import read_pdf  # správný import

# 1) Cesty
INPUT_DIR  =  r'C:\Users\ijttr\OneDrive\Dokumenty\PROG\PYTHON\DATA_ANALYSIS\PD'
OUTPUT_DIR = INPUT_DIR
os.makedirs(OUTPUT_DIR, exist_ok=True)


# ====== Regex pro extrakci ======
# ^([A-Z])   : sekce písmeno
# [.\-]?     : volitelně tečka nebo pomlčka
# (\d{1,3})  : číslo místnosti 1–3 číslic
# \s+        : mezera
# (.+?)      : účel místnosti (lenivě, dokud nenarazí na ...)
# \s+        : mezera
# (\d{1,4},\d{1,2})$ : výměra (1–4 číslic, čárka, 1–2 číslice) a konec řádku
LINE_RX = re.compile(
    r'^([A-Z])[.\-]?(\d{1,3})\s+(.+?)\s+(\d{1,4},\d{1,2})$',
    re.MULTILINE
)

def extract_rooms_from_pdf(pdf_path):
    """Vytáhne všechny řádky legendy místností z PDF jako seznam [kód, účel, výměra]."""
    text_chunks = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            t = page.extract_text()
            if t:
                text_chunks.append(t)
    full_text = "\n".join(text_chunks)
    
    rows = []
    for m in LINE_RX.finditer(full_text):
        sect, num, purpose, area = m.groups()
        code = f"{sect.upper()}.{int(num):03d}"
        purpose = re.sub(r'\s+', ' ', purpose).strip()
        rows.append([code, purpose, area])
    return rows

def process_pdf(pdf_path):
    """Zpracuje jedno PDF, vytvoří DataFrame a uloží do XLSX."""
    pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]
    rows = extract_rooms_from_pdf(pdf_path)
    
    if not rows:
        print(f"⚠️  {pdf_name}: nenašel jsem žádné řádky legendy.")
        return
    
    df = pd.DataFrame(rows, columns=['Číslo místnosti','Účel místnosti','Výmera (m²)'])
    
    # Seřadíme podle sekce a čísla
    df[['Sekce','Číslo']] = df['Číslo místnosti'].str.split('.', expand=True)
    df['Číslo'] = df['Číslo'].astype(int)
    df = df.sort_values(['Sekce','Číslo']).drop(columns=['Sekce','Číslo'])
    
    # Ujistíme se, že výměra je formát x,y
    df['Výmera (m²)'] = df['Výmera (m²)'].map(lambda x: 
        f"{float(x.replace(',','.')):.1f}".replace('.',',')
    )
    
    out_path = os.path.join(OUTPUT_DIR, f"{pdf_name}.xlsx")
    df.to_excel(out_path, index=False)
    print(f"✅ Uloženo: {out_path}")

if __name__ == '__main__':
    for fname in os.listdir(INPUT_DIR):
        if fname.lower().endswith('.pdf'):
            process_pdf(os.path.join(INPUT_DIR, fname))
    print("🎉 Hotovo – všechny Excel soubory jsou ve složce:", OUTPUT_DIR)


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


⚠️  dur_nkb_d01_01_4-3pp_230600: nenašel jsem žádné řádky legendy.


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


⚠️  dur_nkb_d01_02_2pp_230600: nenašel jsem žádné řádky legendy.


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


⚠️  dur_nkb_d01_03_1pp_230600: nenašel jsem žádné řádky legendy.


CropBox missing from /Page, defaulting to MediaBox


⚠️  dur_nkb_d01_04_a1pp_230600: nenašel jsem žádné řádky legendy.
🎉 Hotovo – všechny Excel soubory jsou ve složce: C:\Users\ijttr\OneDrive\Dokumenty\PROG\PYTHON\DATA_ANALYSIS\PD
